# Azure AI Agent service - Single agent

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

## Import required libraries

In [ ]:
import os, time
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ConnectionType
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import ListSortOrder
from dotenv import load_dotenv


load_dotenv()


True

## TEST connection to AI FOUNDRY and get all connections to the default project

In [3]:

endpoint = os.environ["PROJECT_ENDPOINT"]
connection_name = os.environ["CONNECTION_NAME"]

with DefaultAzureCredential(exclude_interactive_browser_credential=False) as credential:

    with AIProjectClient(endpoint=endpoint, credential=credential) as project_client:

        # [START connections_sample]
        print("List all connections:")
        for connection in project_client.connections.list():
            print(connection)

        print("List all connections of a particular type:")
        for connection in project_client.connections.list(
            connection_type=ConnectionType.AZURE_OPEN_AI,
        ):
            print(connection)

        # [END connection_sample]

List all connections:
{'name': 'appInsights-connection-0601', 'id': '/subscriptions/7a28b21e-0d3e-4435-a686-d92889d4ee96/resourceGroups/AI-FOUNDRY-RG/providers/Microsoft.CognitiveServices/accounts/aq-ai-foundry-Sweden-Central/projects/firstProject/connections/appInsights-connection-0601', 'type': 'AppInsights', 'target': '/subscriptions/7a28b21e-0d3e-4435-a686-d92889d4ee96/resourceGroups/AI-FOUNDRY-RG/providers/microsoft.insights/components/aq-APPS-INSIGHTS', 'isDefault': True, 'credentials': {'type': 'ApiKey'}, 'metadata': {'ApiType': 'Azure', 'ResourceId': '/subscriptions/7a28b21e-0d3e-4435-a686-d92889d4ee96/resourceGroups/AI-FOUNDRY-RG/providers/microsoft.insights/components/aq-APPS-INSIGHTS'}}
{'name': 'aqaoaiSWEDENCENTRAL', 'id': '/subscriptions/7a28b21e-0d3e-4435-a686-d92889d4ee96/resourceGroups/AI-FOUNDRY-RG/providers/Microsoft.CognitiveServices/accounts/aq-ai-foundry-Sweden-Central/projects/firstProject/connections/aqaoaiSWEDENCENTRAL', 'type': 'AzureOpenAI', 'target': 'https:/

In [ ]:
#%pip install --upgrade azure-ai-projects

In [ ]:
# [START create_agents_client]
agents_client = AgentsClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

# [END create_agents_client]

with agents_client:

    # [START create_agent]

    agent = agents_client.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],
        name="my-agent",
        instructions="You are helpful agent",
    )
    # [END create_agent]
    print(f"Created agent, agent ID: {agent.id}")

    # [START create_thread]
    thread = agents_client.threads.create()
    # [END create_thread]
    print(f"Created thread, thread ID: {thread.id}")

    # List all threads for the agent
    # [START list_threads]
    threads = agents_client.threads.list()
    # [END list_threads]

    # [START create_message]
    message = agents_client.messages.create(thread_id=thread.id, role="user", content="Hello, tell me a joke")
    # [END create_message]
    print(f"Created message, message ID: {message.id}")

    # [START create_run]
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id)

    # Poll the run as long as run status is queued or in progress
    while run.status in ["queued", "in_progress", "requires_action"]:
        # Wait for a second
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)
        # [END create_run]
        print(f"Run status: {run.status}")

    if run.status == "failed":
        print(f"Run error: {run.last_error}")

    agents_client.delete_agent(agent.id)
    print("Deleted agent")

    # [START list_messages]
    messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role}: {last_text.text.value}")
    # [END list_messages]

    print(thread)

Created agent, agent ID: asst_C0EWgkQblKhLjVDqSkNZrqLz
Created thread, thread ID: thread_3YYilOLH1K7HD6xYmOzcDSpo
Created message, message ID: msg_qF8AFkFZ16mlU3NMUmQ0z3St
Run status: RunStatus.COMPLETED
Deleted agent
MessageRole.USER: Hello, tell me a joke
MessageRole.AGENT: Sure! How about this one:

Why don’t skeletons fight each other?

They don’t have the guts!


In [8]:
thread

{'id': 'thread_3YYilOLH1K7HD6xYmOzcDSpo', 'object': 'thread', 'created_at': 1749237846, 'metadata': {}, 'tool_resources': {}}

In [10]:
message

{'id': 'msg_qF8AFkFZ16mlU3NMUmQ0z3St', 'object': 'thread.message', 'created_at': 1749237846, 'assistant_id': None, 'thread_id': 'thread_3YYilOLH1K7HD6xYmOzcDSpo', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'Hello, tell me a joke', 'annotations': []}}], 'attachments': [], 'metadata': {}}

In [ ]:
run

{'id': 'asst_C0EWgkQblKhLjVDqSkNZrqLz', 'object': 'assistant', 'created_at': 1749237846, 'name': 'my-agent', 'description': None, 'model': 'gpt-4.1', 'instructions': 'You are helpful agent', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}

In [14]:
run.usage

{'prompt_tokens': 29, 'completion_tokens': 25, 'total_tokens': 54, 'prompt_token_details': {'cached_tokens': 0}}

In [18]:
start = run.started_at
end = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  " f"({elapsed.total_seconds():.3f} seconds)")

Start   : 2025-06-06 19:24:08+0000
End     : 2025-06-06 19:24:09+0000
Elapsed : 0:00:01  (1.000 seconds)


## Another agent

In [24]:
# [START create_agents_client]
agents_client = AgentsClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

# [END create_agents_client]

with agents_client:

    # [START create_agent]

    agent = agents_client.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],
        name="my-agent",
        instructions="You are helpful agent",
    )
    # [END create_agent]
    print(f"Created agent, agent ID: {agent.id}")

    # [START create_thread]
    thread = agents_client.threads.create()
    # [END create_thread]
    print(f"Created thread, thread ID: {thread.id}")

    # List all threads for the agent
    # [START list_threads]
    threads = agents_client.threads.list()
    # [END list_threads]

    # [START create_message]
    message = agents_client.messages.create(thread_id=thread.id, role="user", content="What is GenAI")
    # [END create_message]
    print(f"Created message, message ID: {message.id}")

    # [START create_run]
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id)

    # Poll the run as long as run status is queued or in progress
    while run.status in ["queued", "in_progress", "requires_action"]:
        # Wait for a second
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)
        # [END create_run]
        print(f"Run status: {run.status}")

    if run.status == "failed":
        print(f"Run error: {run.last_error}")

    agents_client.delete_agent(agent.id)
    print("Deleted agent")

    # [START list_messages]
    messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role}: {last_text.text.value}")
    # [END list_messages]

    print(thread)

Created agent, agent ID: asst_Tjgvjr958kisuYLw5Qqtqzls
Created thread, thread ID: thread_rqi0Xmn5cptVtCVvdG7THAsz
Created message, message ID: msg_gA7YJ24D2U2RnV1wgqrzkSLO
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.COMPLETED
Deleted agent
MessageRole.USER: What is GenAI
MessageRole.AGENT: **GenAI** stands for **Generative Artificial Intelligence**.

It refers to a category of artificial intelligence models and systems that can generate new content—such as text, images, audio, code, or other data—rather than simply analyzing or recognizing existing data.

### Key Characteristics of GenAI:

- **Content Creation:** GenAI creates new, original content based on patterns it has learned from large datasets. For example, it can write essays, compose music, generate artwork, or write software code.
- **Models:** Popular examples of GenAI models include OpenAI’s GPT (for text), DALL-E (for images), Google’s Imagen, and StabilityAI’s Stable Diffusion.
- **Capabilities:** GenAI can:
 

In [ ]:
run.usage

{'prompt_tokens': 27, 'completion_tokens': 331, 'total_tokens': 358, 'prompt_token_details': {'cached_tokens': 0}}

In [29]:
start = run.started_at
end = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  " f"({elapsed.total_seconds():.3f} seconds)")

Start   : 2025-06-06 19:36:55+0000
End     : 2025-06-06 19:36:58+0000
Elapsed : 0:00:03  (3.000 seconds)
